In [1]:
using DataFrames
using DataFramesMeta
using Plots

In [2]:
"""
# Detect on signals

The FreematicsONE records a '#' in the first column (:x1) when the device turns on.
"""
detect_on_signals(x) = contains(string(x), "#") ? 0 : x

detect_on_signals

In [3]:
# File IO
raw = readcsv("data/01271255.CSV", comments=false)
data = DataFrame(raw)

data[:x1] = detect_on_signals.(data[:x1]);

In [4]:
data = transform(data,
    group = cumsum(data[:x1] .> 0),
    on_seconds = cumsum(data[:x1] / 1000),
    indicator = data[:x4] .!= "");

In [5]:
# Handle ACC, GYR, and MAG x,y,z
data = stack(data, [:x3, :x4, :x5]);

In [6]:
data[:x2] = string.(data[:x2])
head(data)

,variable,value,x1,x2,group,on_seconds,indicator
1,x3,19877,0,131,0,0.0,false
2,x3,742,0,10C,0,0.0,false
3,x3,1,0,10D,0,0.0,false
4,x3,13,0,111,0,0.0,false
5,x3,47,0,104,0,0.0,false
6,x3,81,0,12F,0,0.0,false


In [7]:
# Sort
data = @orderby(data, :group, :variable);

In [8]:
# Filter empty messages
data = @where(data, :value .!= "");

In [9]:
# Relabel :variable
data[data[:variable] .== :x3, :variable] = "x";
data[data[:variable] .== :x4, :variable] = "y";
data[data[:variable] .== :x5, :variable] = "z";

In [10]:
# Rename columns
rename!(data, Dict(:x2=>:data_type) );

In [11]:
# Make unique
data[ (data[:variable] .== :x) .& (data[:data_type] .== "ACC"), :data_type] = "ACC_x"
data[ (data[:variable] .== :y) .& (data[:data_type] .== "ACC"), :data_type] = "ACC_y"
data[ (data[:variable] .== :z) .& (data[:data_type] .== "ACC"), :data_type] = "ACC_z";

In [12]:
"""
Checks whether the GPS time message is valid
"""
function validate_utc(msg)
    #[!ismatch(r"^(?:(?:([01]?\d|2[0-3]))([0-5]?\d))([0-5]?\d)(\d[0])$", string(i)) for i in msg]
    ismatch.(r"^(?:(?:([01]?\d|2[0-3]))([0-5]?\d))([0-5]?\d)(\d[0])$", string.(msg)) |> !
end

validate_utc

In [13]:
# Drop :x1, :variable
delete!(data, [:x1, :variable]);

In [14]:
# Reorder columns
data = @select(data, :on_seconds, :group, :data_type, :value);

In [15]:
# Remove bad GPS data
# UTC should be HHMMSSm0
bad_utc = @where(data, :data_type .== "UTC", validate_utc(:value));

In [18]:
@where(data, x=:data_type .== "LAT", y= :value .< 0)

LoadError: [91mMethodError: no method matching getindex(::DataFrames.DataFrame, ::DataArrays.DataArray{Any,1}, ::Colon)[39m

In [51]:
bad_gps = @where(data, :data_type .== "LAT", isless.(0, :value))
# @where(bad_gps, :value .> 0)

LoadError: [91mMethodError: no method matching getindex(::DataFrames.DataFrame, ::DataArrays.DataArray{Any,1}, ::Colon)[39m

In [32]:
data = join(data, bad_gps, on = :group, kind = :anti)

LoadError: [91mUndefVarError: bad_gps not defined[39m

In [52]:
# Find the reference time
ref_time = @where(data, :data_type .== "UTC")[1,:] |> x->@select(x, :value)
ref_offset = @where(data, :data_type .== "UTC")[1,:] |> x->@select(x, :on_seconds)
data = @transform(data, relative_offset = :on_seconds - ref_offset[1,1])
data = @transform(data, real_time = :relative_offset*100 + ref_time[1,1])
unstack(data, :data_type, :value)

julia(84696,0x110b763c0) malloc: *** mach_vm_map(size=261027979591684096) failed (error code=3)
*** error: can't allocate region
*** set a breakpoint in malloc_error_break to debug


LoadError: [91mOutOfMemoryError()[39m

In [19]:
# wide_data = unstack(data, :group, :data_type, :value)
head(wide_data)

LoadError: [91mUndefVarError: wide_data not defined[39m

In [20]:
data

,on_seconds,group,data_type,value
1,0.0,0,131,19877
2,0.0,0,10C,742
3,0.0,0,10D,1
4,0.0,0,111,13
5,0.0,0,104,47
6,0.0,0,12F,81
7,0.557,1,24,1460
8,0.557,1,20,-9
9,0.557,1,20,1
10,0.557,1,20,-97


In [ ]:
Plots.plot(x=wide_data[:UTC], y=wide_data[:SPD])

In [25]:
q = @from i in data begin
    if i.data_type == "UTC"
        @where i.data_type == "UTC"
    end
    @collect DataFrame
    end

,on_seconds,group,data_type,value
1,3.218,2,UTC,16141220
2,3.348,4,UTC,16141230
3,3.4789999999999996,6,UTC,16141240
4,3.6679999999999997,9,UTC,16141260
5,3.8,11,UTC,16141270
6,3.9699999999999998,13,UTC,16141290
7,4.19,15,UTC,16141310
8,4.3500000000000005,17,UTC,16141330
9,4.489000000000001,19,UTC,16141340
10,4.648000000000001,22,UTC,16141360
